**DTS - Complete Streets** *Right-of-Way Widths for Planned Street Improvements*



# 08 - Modal Priortization

**Author:** rmangan

---

**Purpose:**

Implement Modal Priortizaiton logic for street segments that have Unconstrained Modal Width > ROW Width estimate. 

**This script performs the following functions:**

1. Pre-process data - add all necessary fields, copy unconstrained values to new constrained fields

2. Process Boulevards

3. Process Avenues

4. Process Main Streets

5. Process Streets

6. Process Residential Streets/Lanes/Alleys

7. Calculate Constrained Metrics


**Global Assumptions and Notes:**
1. none

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - geo-processing

In [ ]:
# import modules
import arcpy
import os

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

### Functions

In [ ]:
#Calculate Constrained Modal width
def calc_constrained(cs_type):
    #calculate constrained modal width field
    
    fields = ["SEGMENTID",                 #0
              "modal_width_ped_const",     #1
              "modal_width_bike_const",    #2
              "modal_width_auto_const",    #3
              "modal_width_bus_const",     #4
              "modal_width_park_const",    #5
              "modal_width_medians_const", #6
              "modal_width_const",         #7
              "modal_area_const",          #8
              "modal_area_const_diff",     #9
              "Fac_Type_BP",               #10
              "Fac_Type_BR",               #11
              "NewTier"]                   #12
 
    count = 0
    err_count = 0
    where_clause = ('"cs_type"= \'{}\''.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #igonore shared road-way widths from bike width
                if ((row[10] == "Shared Roadway") or
                    (row[11] == "Shared Roadway")):     
                    width = sum(filter(None,(row[1], row[3], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #ignore bike shared use path for ped > 7    
                elif (((row[10] == "Shared Use Path") or
                      (row[11] == "Shared Use Path")) and
                      (row[1] is not None) and
                      (row[1] > 7)):
                    
                    width = sum(filter(None,(row[1], row[3], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
                elif (((row[10] == "Shared Use Path") or
                      (row[11] == "Shared Use Path")) and
                      (row[1] is not None) and
                      (row[1] < 7)):
                    
                    width = sum(filter(None,(row[1], 7,  row[3], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                #summarize all other segments
                else:  
                    width = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                cursor.updateRow(row)
            
            except (ValueError,TypeError) as error:
                err_count += 1
                print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                
    print("{0} records updated, {1} error(s)".format(count,err_count))

In [ ]:
def remove_parking(cs_type):
    #set constrained modal values for parking to 0 for a given street type
    #accept cs_type as an input street type variable
             
    #fields to expose to update cursor            
    fields = ["modal_width_park_const",   #0
              "modal_width_const"]        #1  
        
    where_clause = ('"cs_type"= \'{}\' AND' 
                    '"modal_width_park_const" > 0 AND'
                    '(NOT "modal_width_park_const" IS NULL) AND'
                    '("modal_width_const" > "row_width_future")'.format(cs_type))
    count = 0
    err_count = 0
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 0    
                count += 1
                cursor.updateRow(row)
            except ValueError as error:
                err_count += 1
                print(error)
                        
    print("{0} records updated, {1} error(s)".format(count,err_count))

In [ ]:
def reduce_bike(cs_type):
    #Reduce Proposed Bike widths to minimum if present, set parallel route evaluation flag to 1
    #Only processes streets where parking is not availalble (see where clause in update cursor)

    
    #fields to expose to update cursor
    fields = ["Fac_Type_BP",            #0
              "Fac_Type_BR",            #1
              "modal_width_bike_const", #2
              "bike_parallel_route"]    #3
         
    where_clause = ('"cs_type" = \'{0}\' AND '
                    'NOT ("Fac_Type_BP" IS NULL OR "Fac_Type_BR" IS NULL) AND '
                    '("modal_width_const" > "row_width_future") AND '
                    '("modal_width_park_const" = 0 OR ("modal_width_park_const" IS NULL))'.format(cs_type))
    count = 0
    err_count = 0
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #Buffered Bike Lanes - Set to minimum 8' width and flag for parallel route evaluation
                if (row[0] == "Buffered Bike Lane" or # 10/15/21 - change to 6.5?
                    row[1] == "Buffered Bike Lane"):  # 10/15/21 - change to 6.5?
                    row[2] = 8
                    row[3] = 1
                
                #Protected Bike Lanes - Set to minimum 10' width and flag for parallel route evaluation 
                elif (row[0] == "Protected Bike Lane" or
                      row[1] == "Protected Bike Lane"):
                    row[2] = 10
                    row[3] = 1
            
                #All other bike features - Set to 5' width and flag for parallel route evaluation
                elif (row[0] == "Bike Lane" or
                      row[0] == "Climbing Lane" or
                      row[0] == "Shoulder Bikeway" or
                      row[0] == "Shared Use Path" or
                      row[0] == "Shared Roadway" or
                      row[1] == "Bike Lane" or
                      row[1] == "Climbing Lane" or
                      row[1] == "Shoulder Bikeway" or
                      row[1] == "Shared Use Path" or
                      row[1] == "Shared Roadway"):
                    row[2] = 5
                    row[3] = 1
                
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))      

In [ ]:
def bike_ped(cs_type):
    #reduce constrained modal widths for bike facilites to 0. (Bike facilities will be placed w/ ped faciliites)
    
    count = 0
    
    #fields to expose to update cursor
    fields = ["SEGMENTID",               #0
              "FULLNAME",                #1
              "modal_width_bike_prop",   #2 - Modal Width - Bike (Proposed - Unconstrained)
              "modal_width_bike_const",  #3 - Modal Width - Bike (Proposed - Constrained)
              "modal_width_ped_const"]   #4 - Modal Width - Ped (Proposed - Constrained)
         
    where_clause = ('"cs_type" = \'{0}\' AND '
                    '("modal_width_const" > "row_width_future") AND '
                    '((NOT "Fac_Type_BE" IS NULL) OR (NOT "Fac_Type_BP" IS NULL) OR (NOT "Fac_Type_BR" IS NULL))'.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("SegmentID: {0}, Name: {1}, Ped Const: {2}, Bike Proposed: {3}".format(row[0], row[1], row[4], row[2])) #use to verify cursor has valid query
            try:
                row[3] = 0
                
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))    

In [ ]:
def cs_flag(cs_type):
#Set Street Type re-evaluation flag for constrained streets
          
    #fields to expose to update cursor
    fields = ["cs_type_revise"] #0
    
    where_clause = ('"cs_type" = \'{0}\' AND' 
                    '("modal_width_const" > "row_width_future")'.format(cs_type))
    count = 0
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 1   
                cursor.updateRow(row)
                count += 1           
            except ValueError as error:
                print(error)
                                          
    print("Done - {0} records updated".format(count))

In [ ]:
def reduce_bus_stops(cs_type):
#reduce ped widths to 8 if Oneway, 16 if Twoway

    #fields to expose to update cursor
    fields = ["SEGMENTID",              #0
              "FULLNAME",               #1
              "ONEWAY",                 #2   
              "NewTier",                #3
              "modal_width_ped_const",  #4
              "modal_width_park_const", #5
              "modal_width_const_desc"] #6

    where_clause = ('"cs_type" = \'{0}\' AND (NOT "NewTier" IS NULL) AND ("modal_width_const" > "row_width_future")'.format(cs_type))
    count = 0
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("SegmentID: {0}, Name: {1}, AB_Lane: {2}, BA_Lane: {3}, VOC: {4}".format(row[0], row[1], row[3], row[4], row[8])) #use to verify cursor has valid query
            try:
                if (row[2] in [1,2]) and row[4] > 8:
                    row[4] = 8                 
                elif row[2] == 0 and row [4] > 16:
                    row[4] = 16
                cursor.updateRow(row)
                count += 1      
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))    

# 01 - Pre-process data


---

Add all necessary fields for data procesing. Copy unconstrained modal width and area values into newly added constrained modal fields. Constrained fields will then be updated by subsquent processing sections. This will enable comparison between Unconstrained and Constrained modal values.

**This section performs the following functions:**

1. Add all necessary fields
2. Copy unnconstrained modal values into new constrained fields (to be updated in later sections)

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

### add required fields

In [ ]:
#01 - add necessary fields & write all modal values to new fields
print("Adding new fields...")
#store field, field alias in dict
new_fields = {"modal_width_ped_const":      "Modal Width - Ped Constrained",
              "modal_width_bike_const":     "Modal Width - Bike Constrained",
              "modal_width_auto_const":     "Modal Width - Auto Constrained",
              "modal_width_bus_const":      "Modal Width - Bus Constrained",
              "modal_width_park_const":     "Modal Width - Parking Constrained",
              "modal_width_medians_const":  "Modal Width - Medians Constrained",
              "modal_width_const":          "Modal Width - Total Constrained",
              "modal_area_const":           "Modal Area - Total Constrained (sq. ft.)",
              "modal_area_diff_const":      "Modal Area - Total Constrained Exceedence (sq. ft.)",
              "bike_parallel_route":        "Bike - Evaluate Parallel Routes",
              "lane_count_rev":             "Lane Count (Constrained)",
              "cs_type_revise":             "Complete Street Type - Re-Evaluate"}

#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

#loop through dict and add fields if they are not currently in the dataset
for key,value in new_fields.items():
    if key in field_names:
        print("{0} field already present, skipping".format(key))
    elif key in ("modal_area_const","modal_area_diff_const"):
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="FLOAT", field_alias = value)
    else:
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="SHORT", field_alias = value)
          
print("Field additions complete")
print("\n")

### copy unconst values to const fields

In [ ]:
#write all previous proposed modal values to new fields (values will be altered in subsquent processing)

#fields to expose to update cursor
fields = ["modal_width_auto_prop",    #0
          "modal_width_auto_const",   #1
          "modal_width_bike_prop",    #2
          "modal_width_bike_const",   #3
          "modal_width_bus_prop",     #4
          "modal_width_bus_const",    #5
          "modal_width_park_prop",    #6
          "modal_width_park_const",   #7
          "modal_width_ped_prop",     #8
          "modal_width_ped_const",    #9
          "modal_width_max",          #10
          "modal_width_const",        #11
          "modal_area_max" ,          #12     
          "modal_area_const",         #13
          "modal_width_medians_prop", #14
          "modal_width_medians_const" #15
         ]

print("Copying all Unconstrained values to Constrained fields for initial processing...")
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[1] = row[0]
            row[3] = row[2]
            row[5] = row[4]
            row[7] = row[6]
            row[9] = row[8]
            row[11] = row[10]
            row[13] = row[12]
            row[15] = row[14]

            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            print(error)
            #err_count += 1
            #print("Error at SegmentID: {0} - {1} ".format(row[0],error))          

print("Done - Constrained Fields calculated from Unconstrained")

In [ ]:
print("Adding new fields...")
#store field, field alias in dict
new_fields = {
    "row_width_future":      "ROW Width Future (greater of estimate & 1986 proposed)"
}

#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

#loop through dict and add fields if they are not currently in the dataset
for key,value in new_fields.items():
    if key in field_names:
        print("{0} field already present, skipping".format(key))
    elif key in ("row_width_future"):
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="FLOAT", field_alias = value)
    else:
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="SHORT", field_alias = value)
          
print("Field additions complete")

In [ ]:
#calc row width field for prioritization (max of current & 1986 prop)
#fields to expose to update cursor
fields = [
    "mean_row_2020",   #0
    "row_prop_num",    #1
    "row_width_future" #2
]
count = 0
err_count = 0
print("Calc max of mean vs 1986 prop...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[1] is not None) and
               row[1] > row[0]):
                row[2] = row[1]
                count += 1
            else:
                row[2] = row[0]    
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            print(error)
            err_count += 1
            #print("Error at SegmentID: {0} - {1} ".format(row[0],error))          

print("{0} records updated, {1} error(s)".format(count,err_count))



# Modal Prioritization

## 02 - Boulevards
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Documentation%20Images/Code/08_Modal_Prioritization/Boulevard.jpg" alt="Logic flow diagram for Boulevards" style="height: 200px;"/>

**This section performs the following functions for all Boulevard Street Types:**

1. If ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Minimum Modal widths for Bike Facilities used:
   1. Buffered Bike Lane - 8'
   2. Protected Bike Lane - 10'
   3. Bike Lane, Climbing Lane, Shoulder Bikeway, Shared Use Path, Shared Roadway - 5'

In [ ]:
print("Processing Boulevards....")

## 01 - remove parking
print("02.01 - removing parking...\n")
remove_parking("Boulevard")

## 02 - Re-Calcualte Constrained Modal Width
print("02.02 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")

## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1
print("02.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Boulevard")

## 04 - Re-Calcualte Constrained Modal Width for Boulevards
print("02.04 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")

## 05 - Set all remaining constrained widths that are boulevards to street type re-evaluation
print("02.05 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")

print("Boulevards Complete")

## 03 - Avenues
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Documentation%20Images/Code/08_Modal_Prioritization/Avenue.jpg" alt="Logic flow diagram for Avenues" style="height: 200px;"/>

**This section performs the following functions for all Avenue Street Types:**

1. If Street Type = Avenue, ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width field
3. If Street Type = Avenue, ROW = Constrained, and parking = 0 or NULL, and VOC <= 0.8, reduce auto transit lane count.
4. Re-calcualte Constrained Modal width field
5. If Street Type = Avenue, ROW = Constrained, parking = 0 or NULL, VOC > 0.8, reduce sidewalk facilities to minimum width
6. Re-calcualte Constrained Pedestrian width
7. Re-calculalate Constrained Modal width fields
8. If still constrained, set flag for street type re-evaluation


**Assumptions and Notes:**
1. Streets must have a minimum of 3 lanes to be eligible for lane count reduction, regardless of one-way/two-way classification
2. All streets have lanes counts reduced by only 1 (regardless of lane count)
3. Existing Ped Facilites from sidealk inventory are not reduced in size. Only planned ped improvements are reduced.

In [ ]:
print("Processing Avenues....\n")

# Reduce Bus Stop Areas
print("03.01 - Reduce bus stop areas...")
reduce_bus_stops("Avenue")

## Re-Calculate Constrained Modal Width
print("03.02 - Re-calculate constrained width values")
calc_constrained("Avenue")

# Remove Parking
print("03.03 - removing parking...")
remove_parking("Avenue")

## Re-Calculate Constrained Modal Width
print("03.04 - Re-calculate constrained width values")
calc_constrained("Avenue")

In [ ]:
## 03 - Reduce lane count by 1 for all segments & reduce auto modal width by 12 (assume outside lane removed)
print("03.05 - Remove Auto lanes for Streets <= 0.8 VOC")
  
#fields to expose to update cursor
fields = ["SEGMENTID",              #0
          "FULLNAME",               #1
          "ONEWAY",                 #2   
          "lane_count_exist",       #3
          "lane_count_prop",        #4
          "lane_count_rev",         #5
          "modal_width_auto_const", #6     
          "VOC_max"]                #7

count = 0         
where_clause = ('"cs_type" = \'Avenue\' AND'
                ' ("modal_width_const" > "row_width_future") AND'
                ' NOT ("VOC_max" IS NULL) AND'
                ' "VOC_max" <= 0.8 AND'
                ' NOT ("lane_count_exist" IS NULL) AND'
                ' ("lane_count_exist" >= 3)')
    
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("SegmentID: {0}, Name: {1}, AB_Lane: {2}, BA_Lane: {3}, VOC: {4}".format(row[0], row[1], row[3], row[4], row[8])) #use to verify cursor has valid query
        try:
            lane_current = row[3]
            lane_revision = row[3] - 1
            auto_current = row[6]
            auto_revised = (row[6] - 12)
            
            row[5] = lane_revision
            row[6] = auto_revised
                
            cursor.updateRow(row)
            count += 1
        
        except ValueError as error:
            print(error)

print("Done - {0} records updated".format(count))    

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("03.06 - Re-calculate constrained width values")
calc_constrained("Avenue")

In [ ]:
## 05 - Reduce sidewalk widths to minimum
print("03.07 - Reduce sidewalk widths to minimum")

count = 0
    
#fields to expose to update cursor
fields = ["SEGMENTID",              #0
          "FULLNAME",               #1
          "modal_width_ped_exist",  #2 - Modal Width - Ped (Existing)
          "modal_width_ped_prop",   #3 - Modal Width - Ped (Proposed)
          "modal_width_ped_const"]  #4 - Modal Width - Ped (Constrained)
         
where_clause = ('"cs_type" = \'Avenue\' AND'
                ' ("modal_width_const" > "row_width_future") AND'
                ' ((NOT "FID_Ped_Add_temp" IS NULL) OR (NOT "FID_Ped_Improve_temp" IS NULL))')

with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("SegmentID: {0}, Name: {1}, Ped Uncon: {2}, Ped Con: {3}".format(row[0], row[1], row[2], row[3])) #use to verify cursor has valid query
        try:
            #change all proposed sidewalk modal values to 8' total ped width value if Unconstrained was increased due to Ped Add/Ped Improve
            if (
                (row[3] is not None) and
                (row[2] is not None) and
                (row[3] > row[2])
            ):
                row[4] = 8
                count += 1
        
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)

print("Done - {0} records updated".format(count))    

In [ ]:
## 06 - Re-Calculate Constrained Modal Width
print("\n03.08 - Re-calculate constrained width values")
calc_constrained("Avenue")

## 07 - Set all remaining constrained widths to street type re-evaluation
print("\n03.09 - Set street type re-evaluation flage for remaining constrained segments")
cs_flag("Avenue")

print("\nAvenues Complete")

## 04 - Main Streets
---

Calculate Modal Priortization logic and revised Modal widths for Main Strets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Documentation%20Images/Code/08_Modal_Prioritization/MainStreet.jpg" alt="Logic flow diagram for Main Streets" style="height: 200px;"/>

**This section performs the following functions for all 'Main Street' Street Types:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Combingin bike & pedestrain facilites is done by...
2. All streets have lanes counts reduced by only 1 (regardless of lane count)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
print("Processing Main Streets....")

## 01 - Combine Bike & Pedestrian Facilities
print("\n04.01 - Combining Bike & Pedestrian facilities...")
bike_ped("Main Street")

## 02 - Re-Calculate Constrained Modal Width
print("\n04.02 - Re-calculate constrained width values")
calc_constrained("Main Street")

## 03 - Remove parking
print("\n04.03 - removing parking...")
remove_parking("Main Street")

## 04 - Re-Calculate Constrained Modal Width
print("\n04.04 - Re-calculate constrained width values")
calc_constrained("Main Street")

## 05 - Set all remaining constrained widths to street type re-evaluation
print("\n04.05 - Set street type re-evaluation flage for remaining constrained segments")
cs_flag("Main Street")

print("\nMain Streets Complete")

## 05 - Streets
---

Calculate Modal Priortization logic and revised Modal widths for Streets (previously "Major Streets").

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Documentation%20Images/Code/08_Modal_Prioritization/MajorStreet.jpg" alt="Logic flow diagram for Major Streets" style="height: 200px;"/>

**This section performs the following functions for all 'Street' Street Types:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Logic is identical to logic used for Boulevards but maintained as seperate code sections to allow for individual changes if needed
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
print("Processing Streets....\n")

## 01 - Remove parking
print("\n05.01 - removing parking...")
remove_parking("Street")

## 02 - Re-Calculate Constrained Modal Width
print("\n05.02 - Re-calculate constrained width values")
calc_constrained("Street")

## 03 - Reduce Bus Stop areas
print("\n05.03 - removing parking...")
reduce_bus_stops("Street")

## 04 - Re-Calculate Constrained Modal Width
print("\n05.04 - Re-calculate constrained width values")
calc_constrained("Street")

## 05 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1
print("\n05.05 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation")  
reduce_bike("Street")

## 06 - Re-Calculate Constrained Modal Width
print("\n05.06 - Re-calculate constrained width values")
calc_constrained("Street")

## 07 - Set all remaining constrained widths to street type re-evaluation
print("\n05.07 - Set street type re-evaluation flag for remaining constrained segments")
cs_flag("Street")

print("\nStreets Complete")

## 06 - Streets, Lanes, Alleys
---

Calculate Modal Priortization logic and revised Modal widths for Residential Street and Lane/Alley

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Documentation%20Images/Code/08_Modal_Prioritization/Street.jpg" alt="Logic flow diagram for Streets and Lane/Alleys" style="height: 200px;"/>

**This section performs the following functions for all 'Residential Street' and Lane/Alley Street Types:**

1. If ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
6. Re-calcualte Constrained Modal width field
7. If ROW = Constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. This section should be run twice, once with var = "Residential Street", and secondly with var = "Lane/Alley"

In [ ]:
# var = "Residential Street"
var = "Lane/Alley"

print("Processing {}s\n".format(var))

## 01 - Reduce Bus Stop areas
print("\n06.01 - Reduce Bus Stop areas...")
reduce_bus_stops(var)

## 02 - Re-Calculate Constrained Modal Width
print("\n06.02 - Re-calculate constrained width values")
calc_constrained(var)

## 03 - Remove parking
print("\n06.03 - removing parking...")
remove_parking(var)

## 04 - Re-Calculate Constrained Modal Width
print("\n06.04 - Re-calculate constrained width values")
calc_constrained(var)

## 05 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1
print("\n06.05 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation")  
reduce_bike(var)

## 06 - Re-Calculate Constrained Modal Width
print("\n06.06 - Re-calculate constrained width values")
calc_constrained(var)

## 07 - Combine Bike & Pedestrian Facilities
print("\n06.07 - Combining Bike & Pedestrian facilities...")
bike_ped(var)

## 08 - Re-Calculate Constrained Modal Width
print("\n06.08 - Re-calculate constrained width values")
calc_constrained(var)

## 09 - Set all remaining constrained widths to street type re-evaluation
print("\n06.09 - Set street type re-evaluation flag for remaining constrained segments")
cs_flag(var)

print("\n{}s done".format(var))

## 07 - Calculate Constrained Modal Areas

In [ ]:
#fields for cursor

fields = ["SEGMENTID",                #0
          "Shape_Length",             #1
          "length_ft",                #2
          "mean_row_2020",            #3          
          "mean_row_area",            #4
          "modal_width_max",          #5
          "modal_width_max_diff",     #6  
          "modal_width_const",        #7          
          "modal_width_const_diff",   #8  <--
          "modal_area_max",           #9
          "modal_area_max_diff",      #10
          "modal_area_const",         #11 <--
          "modal_area_const_diff",    #12 <--
          "lane_count_rev",           #13
          "lane_miles_const",         #14          
          "modal_width_auto_const",   #15
          "modal_area_auto_const",    #16          
          "modal_width_bike_const",   #17
          "modal_area_bike_const",    #18            
          "modal_width_park_const",   #19
          "modal_area_park_const",    #20          
          "modal_width_ped_const",    #21
          "modal_area_ped_const",     #22          
          "modal_width_bus_const",    #23
          "modal_area_bus_const",     #24
          "row_width_future",         #25
          "modal_width_medians_const",#26
          "modal_area_medians_const"  #27
         ]
        
print("\nCalculating total areas..")
count = 0
err_count = 0
where_clause = 'NOT "row_width_future" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            modal_width_const_diff = row[7] - row[3]             #Const Width - ROW width           
            modal_area_const = row[2] * row[7]                   #Const Width x Length
            modal_area_const_diff = modal_area_const - row[4]    #Const Area - ROW Area

            row[8] = modal_width_const_diff
            row[11] = modal_area_const
            row[12] = modal_area_const_diff
            
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Auto
print("\nAuto area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_auto_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[16] = row[2] * row[15] #Auto            
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Bike
print("\nBike area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_bike_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[18] = row[2] * row[17] #Bike         
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Parking
print("\nParking area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_park_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[20] = row[2] * row[19] #Parking         
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Ped
print("\nPed area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_ped_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[22] = row[2] * row[21] #Ped     
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Bus
print("\nBus area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_bus_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[24] = row[2] * row[23] #Bus     
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Medians
print("\nMedian area..")
count = 0
err_count = 0
where_clause = 'NOT "modal_width_medians_const" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause ) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[27] = row[2] * row[26] #Medians     
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("{0} records updated, {1} error(s)".format(count,err_count))


##Lane Miles
count = 0
err_count = 0
print("\nRevised lane miles..")
with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "lane_count_rev" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            row[14] = (row[13]*row[2])/5280            
            count +=1
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                      
print("{0} records updated, {1} error(s)".format(count,err_count))

print("\nComplete.")

In [ ]:
print("Modal Prioritizitation processing complete")